In [ ]:
import socketio
from datetime import datetime
import time
import json
import os
from IPython.display import clear_output

# standard Python

pathUBI=os.path.join(os.getcwd(),"data/ubitrack-uwb")

def getTime():
    return str(int(time.time()))+"\t"+datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def key_file_time():
    return str((int)(time.time()/(60*60*24)))

def key_file_time_now():
    return str((int)(time.time()))

zone_id = '2'
algorithm = '80'
  
print("punn!")
sio = socketio.Client(logger=True, engineio_logger=True)

@sio.event
def connect():
    print("I'm connected!")
    sio.emit('join', algorithm + "_" + zone_id );

@sio.event
def connect_error():
    print("The connection failed!")

    
@sio.event
def message(data):
    print('I received a message!')

@sio.event
def on_message(data):
    print('I received a data,',data)
    
@sio.on('say')
def on_message(data):
    print('Data ', data)


@sio.event
def say(data):
    clear_output(wait=True)
    print('Data say: ', data)
    json_data = json.loads(data)
    if 'tagaddr' in json_data:
        tagaddr=json_data['tagaddr']
        if 'x' in json_data and 'y' in json_data and 'z' in json_data:
            file = open(os.path.join(pathUBI , key_file_time()+"."+tagaddr+".xyz.tsv"), 'a')
            file.write(getTime()+"\t"+str(json_data['x'])+"\t"+str(json_data['y'])+"\t"+str(json_data['z'])+"\n")
            file.close()
        if 'sourcedata' in json_data:
            t0=getTime()
            file = open(os.path.join(pathUBI , key_file_time()+"."+tagaddr+".sourcedata.tsv"), 'a')
            sourcedata=json_data['sourcedata']
            for sdata, value in sourcedata.items():
                file.write(t0+"\t"+str(sdata)+"\t"+str(value)+"\n")
            file.close()
            
        if 'rssi' in json_data:
            t0=getTime()
            file = open(os.path.join(pathUBI , key_file_time()+"."+tagaddr+".rssi.tsv"), 'a')
            rssicedata=json_data['rssi']
            for sdata, value in rssicedata.items():
                file.write(t0+"\t"+str(sdata)+"\t"+str(value)+"\n")
            file.close()            
    
print("hi!")

sio.connect(url='http://192.168.1.8:3000?token=mdpscloudforce')
sio.wait()

print("bye!")




Data say:  {"datatype":80,"pno":"47","type":"normal","tagaddr":"f02e","x":-3.8062652374674375,"y":3.5575205838768973,"tagtime":1710623404326,"zoneid":"2","in_fence":"","sourcedata":{"8171":5.522,"2eff":6.627,"eb38":3.212},"rssi":{"8171":-32,"2eff":-41,"eb38":-47},"match_group":"20"}
